In [ ]:
from google.colab import files
#uploaded = files.upload()

In [ ]:
#!pip -q install gradio
#!pip -q install PySastrawi

In [ ]:
# Standard Library
import string
import re

# Third-party Library
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
import gradio as gr

nltk.download('punkt')
pd.set_option("display.max_colwidth", 1000)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
df = pd.read_csv("review_hotel.csv")

In [ ]:
df.head()

,review_id,review_text,category
0,0fc651bfdd38a3b4ba19ef9d0d1e92e3,"Penginapannya rapi, bersih, suasana tenang, pelayannya ramah dan banyak membantu. Lokasi cukup dekat ke beberapa obyek wisata. kalau saya ke bandung lagi mungkin saya akan menginap lagi di sini.",1
1,7d8cb2e465c5602e51c1ba8710e8ff16,"wifi tidak sampai kamar, tidak tersedia heater untuk kopi/teh atau dispenser dan tidak tersedia kursi di depan kamar untuk merokok hingga harus duduk di lantai",0
2,1f4447c73e5e1aac462d05b0ecbd5a55,"Lantai kamar mandi kotor, bau, dan seperti sdh jdi sarang nyamuk dikamar mandinya...pesan superior double yg dikasih twin...mungkin karena murah saya maklumin, tpi kedepan mohon untuk bisa jdi perbaikan untuk bisa lbh baik lg..",0
3,b9aefbfb0debbfff43f8b268c82b43c0,"Wc jorok.. Kasur tidak dibersihkan,, handuk tidak diganti,",0
4,e2afef18bd695fae43d4d53b8c547bd4,bocor lagi,0


In [ ]:
df.isna().sum()

review_id      0
review_text    0
category       0
dtype: int64

In [ ]:
# Cek emoticon
df["review_text"].str.findall("(:\S+)").explode().value_counts()

:(                         145
:)                          42
:00                         16
:((                          8
:'(                          4
:49                          3
:))                          2
:p                           2
:15                          2
:30                          2
:00,                         2
:v                           2
:00.                         2
:(((                         2
:"                           1
:wastafel                    1
:((..                        1
:di                          1
:09)                         1
:30-20.00                    1
:-)                          1
:kamar                       1
:),                          1
:00an.                       1
://youtu.be/6t2Hqf0-HuU      1
:')                          1
:(((((((                     1
::                           1
:P                           1
:;                           1
:-                           1
:57                          1
:55     

In [ ]:
df["review_text"].str.findall("\w+!").explode().value_counts()

banget!          9
kecewa!          9
mengecewakan!    8
bad!             7
recommended!     7
                ..
gmn!             1
affordable!      1
EVER!            1
parrah!          1
Menyebalkan!     1
Name: review_text, Length: 286, dtype: int64

In [ ]:
punctuations = re.sub(r"[!<_>#:)\.]", "", string.punctuation)

def punct2wspace(text):
    return re.sub(r"[{}]+".format(punctuations), " ", text)

def normalize_wspace(text):
    return re.sub(r"\s+", " ", text)

def casefolding(text):
    return text.lower()

def separate_punct(text):
    return re.sub(r"(\w+)(!)", r"\1 \2", text)

In [ ]:
# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# stemming process
sentence = "Perekonomian Indonesia sedang dalam pertumbuhan yang membanggakan"
output   = stemmer.stem(sentence)

print(output)
# ekonomi indonesia sedang dalam tumbuh yang bangga

print(stemmer.stem("Mereka meniru-nirukannya"))

ekonomi indonesia sedang dalam tumbuh yang bangga
mereka tiru


In [ ]:
def preprocess_text(text):
    text = punct2wspace(text)
    text = normalize_wspace(text)
    text = casefolding(text)
    text = separate_punct(text)
    # text = stemmer.stem(text)
    return text

preprocess_text("maikuru@superop")

'maikuru superop'

In [ ]:
# Ilustrasi teks yang sudah dibersihkan
df["cleaned_text"] = df["review_text"].apply(preprocess_text)
df.head()

,review_id,review_text,category,cleaned_text
0,0fc651bfdd38a3b4ba19ef9d0d1e92e3,"Penginapannya rapi, bersih, suasana tenang, pelayannya ramah dan banyak membantu. Lokasi cukup dekat ke beberapa obyek wisata. kalau saya ke bandung lagi mungkin saya akan menginap lagi di sini.",1,penginapannya rapi bersih suasana tenang pelayannya ramah dan banyak membantu lokasi cukup dekat ke beberapa obyek wisata kalau saya ke bandung lagi mungkin saya akan menginap lagi di sini
1,7d8cb2e465c5602e51c1ba8710e8ff16,"wifi tidak sampai kamar, tidak tersedia heater untuk kopi/teh atau dispenser dan tidak tersedia kursi di depan kamar untuk merokok hingga harus duduk di lantai",0,wifi tidak sampai kamar tidak tersedia heater untuk kopi teh atau dispenser dan tidak tersedia kursi di depan kamar untuk merokok hingga harus duduk di lantai
2,1f4447c73e5e1aac462d05b0ecbd5a55,"Lantai kamar mandi kotor, bau, dan seperti sdh jdi sarang nyamuk dikamar mandinya...pesan superior double yg dikasih twin...mungkin karena murah saya maklumin, tpi kedepan mohon untuk bisa jdi perbaikan untuk bisa lbh baik lg..",0,lantai kamar mandi kotor bau dan seperti sdh jdi sarang nyamuk dikamar mandinya pesan superior double yg dikasih twin mungkin karena murah saya maklumin tpi kedepan mohon untuk bisa jdi perbaikan untuk bisa lbh baik lg
3,b9aefbfb0debbfff43f8b268c82b43c0,"Wc jorok.. Kasur tidak dibersihkan,, handuk tidak diganti,",0,wc jorok kasur tidak dibersihkan handuk tidak diganti
4,e2afef18bd695fae43d4d53b8c547bd4,bocor lagi,0,bocor lagi


In [ ]:
# Kolom teks
cleaned_text = df["cleaned_text"]
cleaned_text.head()

0                                  penginapannya rapi bersih suasana tenang pelayannya ramah dan banyak membantu lokasi cukup dekat ke beberapa obyek wisata kalau saya ke bandung lagi mungkin saya akan menginap lagi di sini 
1                                                                 wifi tidak sampai kamar tidak tersedia heater untuk kopi teh atau dispenser dan tidak tersedia kursi di depan kamar untuk merokok hingga harus duduk di lantai
2    lantai kamar mandi kotor bau dan seperti sdh jdi sarang nyamuk dikamar mandinya pesan superior double yg dikasih twin mungkin karena murah saya maklumin tpi kedepan mohon untuk bisa jdi perbaikan untuk bisa lbh baik lg 
3                                                                                                                                                                         wc jorok kasur tidak dibersihkan handuk tidak diganti 
4                                                                                                   

In [ ]:
count_vect = CountVectorizer(max_features=10_000)
count_repr = count_vect.fit_transform(cleaned_text)
count_repr

<14856x10000 sparse matrix of type '<class 'numpy.int64'>'
	with 213952 stored elements in Compressed Sparse Row format>

In [ ]:
tfidf_vect = TfidfVectorizer(max_features=10_000)
tfidf_repr = tfidf_vect.fit_transform(cleaned_text)
tfidf_repr

<14856x10000 sparse matrix of type '<class 'numpy.float64'>'
	with 213952 stored elements in Compressed Sparse Row format>

In [ ]:
logres = LogisticRegression()
multi_nb = MultinomialNB()

In [ ]:
target = df["category"].map({0 : "negative" , 1 :"positive" })
features = df["cleaned_text"]
target.head()

0    positive
1    negative
2    negative
3    negative
4    negative
Name: category, dtype: object

In [ ]:
train_X, test_X, train_y, test_y = train_test_split(features, target, test_size=0.2, random_state=42)

In [ ]:
# Pipeline Logistic
logres_pipe = Pipeline(
    [
     ("feature_extractions", tfidf_vect),
     ("classifier", logres)
    ]
)

# Pipeline NB
multinb_pipe = Pipeline(
    [
     ("feature_extractions", count_vect),
     ("classifier", multi_nb)
    ]
)

In [ ]:
multinb_pipe.fit(train_X, train_y)
multinb_pipe.score(test_X, test_y)

0.9434724091520862

In [ ]:
logres_pipe.fit(train_X, train_y)
logres_pipe.score(test_X, test_y)

0.9471736204576043

In [ ]:
# Multinomial Naive Bayes
multinb_report = classification_report(y_true=test_y, y_pred=multinb_pipe.predict(test_X))
print(multinb_report)

              precision    recall  f1-score   support

    negative       0.96      0.98      0.97      2589
    positive       0.84      0.69      0.76       383

    accuracy                           0.94      2972
   macro avg       0.90      0.84      0.86      2972
weighted avg       0.94      0.94      0.94      2972



In [ ]:
# Logistic Regression
logres_report = classification_report(y_true=test_y, y_pred=logres_pipe.predict(test_X))
print(logres_report)

              precision    recall  f1-score   support

    negative       0.95      0.99      0.97      2589
    positive       0.93      0.64      0.76       383

    accuracy                           0.95      2972
   macro avg       0.94      0.82      0.86      2972
weighted avg       0.95      0.95      0.94      2972



In [ ]:
import joblib
filename = 'project_nlp.sav'
joblib.dump(logres, filename)

['project_nlp.sav']

In [ ]:
files.download('project_nlp.sav')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>